In [8]:
!wget --inet4-only https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-09-25 12:26:07--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  1.94MB/s    in 0.5s    

2024-09-25 12:26:08 (1.94 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [9]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [10]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [11]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [12]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [17]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [18]:
import torch
data =  torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [20]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [21]:
block_size = 8
train_data[:block_size+1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [23]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when in put is {context} output is: {target}")


when in put is tensor([18]) output is: 47
when in put is tensor([18, 47]) output is: 56
when in put is tensor([18, 47, 56]) output is: 57
when in put is tensor([18, 47, 56, 57]) output is: 58
when in put is tensor([18, 47, 56, 57, 58]) output is: 1
when in put is tensor([18, 47, 56, 57, 58,  1]) output is: 15
when in put is tensor([18, 47, 56, 57, 58,  1, 15]) output is: 47
when in put is tensor([18, 47, 56, 57, 58,  1, 15, 47]) output is: 58


In [29]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) 
    return x,y

xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print ("---------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} target is: {target}")

ix: tensor([ 76049, 234249, 934904, 560986])
inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---------
when input is [24] target is: 43
when input is [24, 43] target is: 58
when input is [24, 43, 58] target is: 5
when input is [24, 43, 58, 5] target is: 57
when input is [24, 43, 58, 5, 57] target is: 1
when input is [24, 43, 58, 5, 57, 1] target is: 46
when input is [24, 43, 58, 5, 57, 1, 46] target is: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] target is: 39
when input is [44] target is: 53
when input is [44, 53] target is: 56
when input is [44, 53, 56] target is: 1
when input is [44, 53, 56, 1] target is: 58
when input is [44, 53, 56, 1, 58

In [59]:
import torch
import torch.nn as nn
from torch.nn import functional as F 

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    

    def forward(self, idx, targets=None):

        logits = self.token_embedding_table(idx)
        loss = None
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
         
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
        


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))



torch.Size([32, 65])
tensor(4.6825, grad_fn=<NllLossBackward0>)

&?IRgBQgRffxCm;wWWXl$gVsLqSf'b !Up;WBr&qK$'WR UcZal
gwl-'XwKhv
JR;QBUcfx&YIgZI$AWFvYLR !FIH.gQQ.tZFh


In [60]:
optimiser = torch.optim.AdamW(m.parameters(), lr=1e-3)


In [61]:
batch_size = 32
for steps in range(100):
    xb, yb = get_batch("train")

    logits, loss = m(xb, yb)
    optimiser.zero_grad(set_to_none=True)
    loss.backward()
    optimiser.step()
    print(loss)

    

ix: tensor([423689, 225135, 282755, 782948, 914763, 458318, 458711, 195912, 271579,
        784883,  74241, 797742, 603452,  16319, 728509, 805059, 964807, 858938,
        870023, 804642, 793964, 122162, 818716, 878864, 113394, 272727, 618854,
        322270, 311818, 677099, 370536, 264883])
tensor(4.6953, grad_fn=<NllLossBackward0>)
ix: tensor([ 368065,  114067,   60822,  120818,  962257,  676071,  875203,  491328,
         707709,  732462,  445144,  458693,  131153,  929610,  385629,  949064,
         655137,  849062,  864301,  532191,  855369,  318858,  645651,  872250,
         133516,  758168,  698318, 1002302,  893537,  239438,  386125,  868203])
tensor(4.5882, grad_fn=<NllLossBackward0>)
ix: tensor([ 879474,  191867,  317281,  522226,  803665,  232155,  136981,  716886,
         720827,  834766, 1002431,  471337,  838198,  940279,  971748,  628385,
         381202,   25304,  881383,  615645,  257321,  374073,   75163,   32883,
         337208,  357649,  374224,  740207,  534361,